## case1

In [ ]:
"""
Case1_main.py

"""

import numpy as np
import pandas as pd
import copy

# -------------------------
# Step 1. Person 類別 (含 region)
# -------------------------
class Person:
    """
    - person_id: 全域唯一編號
    - budget: 剩餘預算
    - capacity: 剩餘可喝量 (ml)
    - person_type: 'A' / 'B' / 'C'
    - price_sensitive: True / False
    - favorite_list: B/C 時的喜好酒款清單；A 時可能為 None
    - beta: 需求函數中對價格的敏感度
    - region: 'north' or 'south' (此Case2增加的欄位)
    """
    def __init__(self, person_id, budget, capacity, person_type, price_sensitive, favorite_list, beta, region):
        self.person_id = person_id
        self.budget = budget
        self.capacity = capacity
        self.person_type = person_type
        self.price_sensitive = price_sensitive
        self.favorite_list = favorite_list
        self.beta = beta
        self.region = region

# -------------------------
# Step 2. 全域參數
# -------------------------
NUM_BEERS = 20
INIT_PRICE = 1.0
UPPER_BOUND = 1.2
LOWER_BOUND = 0.5
PRICE_INCREASE = 0.005
PRICE_DECREASE = 0.04
TIME_SLOTS = 26  # 一天 13 小時，每半小時一個 slot
DAYS = 7

ALPHA = 500
NOISE_STD = 100

P_A = 0.4
P_B = 0.3
P_C = 0.3

P_SENSITIVE = 0.5

# 熱門酒款設定 (假設有5款)
HOT_BEERS = [0, 1, 2, 3, 4]
beer_weights = [3 if i in HOT_BEERS else 1 for i in range(NUM_BEERS)]
weight_sum = sum(beer_weights)

def sample_hot_beers(k):
    probs = np.array(beer_weights) / weight_sum
    chosen = np.random.choice(np.arange(NUM_BEERS), size=k, replace=False, p=probs)
    return chosen.tolist()

def sample_one_hot_beer():
    probs = np.array(beer_weights) / weight_sum
    return np.random.choice(np.arange(NUM_BEERS), p=probs)

# -------------------------
# Step 3. 需求函數
# -------------------------
def demand_function(price, beta):
    noise = np.random.normal(0, NOISE_STD)
    q = ALPHA - beta * price + noise
    return max(0, q)

# -------------------------
# Step 4. 選酒邏輯 (A/B/C + 價格敏感)
# -------------------------
def choose_beer(person, beer_prices):
    ptype = person.person_type
    sensitive = person.price_sensitive
    flist = person.favorite_list
    all_indices = np.arange(NUM_BEERS)

    if ptype == 'B':
        return flist[0]
    if ptype == 'C':
        if sensitive:
            possible_prices = [beer_prices[b] for b in flist]
            best_idx = flist[np.argmin(possible_prices)]
            return best_idx
        else:
            return np.random.choice(flist)

    # ptype == 'A'
    if sensitive:
        inv_prices = 1 / beer_prices
        w = inv_prices / inv_prices.sum()
        return np.random.choice(all_indices, p=w)
    else:
        return np.random.randint(0, NUM_BEERS)


# -------------------------
# Step 5. generate_new_people_for_store
#   根據 store (north / south) 做不同預算 & beta 分布
# -------------------------
def generate_new_people_for_store(base_person_id, num_people, store="north", budget_value=1000):
    persons = []
    current_id = base_person_id

    for _ in range(num_people):
        # 1) 決定三種類型 (A/B/C)
        r = np.random.rand()
        if r < P_A:
            p_type = 'A'
        elif r < P_A + P_B:
            p_type = 'B'
        else:
            p_type = 'C'

        # 2) 決定是否敏感
        is_sensitive = (np.random.rand() < P_SENSITIVE)

        # 3) favorite_list
        if p_type == 'A':
            fav_list = None
        elif p_type == 'B':
            fav_list = [sample_one_hot_beer()]
        else:
            L = np.random.randint(2, 6)
            fav_list = sample_hot_beers(L)

        # 4) 根據 store (北/南) 給不同預算 & beta
        if store == "north":
            # 北部：預算高 + beta 偏低 (不敏感)
            while True:
                budget = np.random.normal(budget_value, 200)  # 平均1200
                if budget > 0:
                    break
            if is_sensitive:
                beta_ = 20 # NOTE 暫時設定為固定值
            else:
                beta_ = 20 # NOTE 暫時設定為固定值
        else:
            # 南部：預算低 + beta 偏高 (比較敏感)
            while True:
                budget = np.random.normal(budget_value, 200)  # 平均800
                if budget > 0:
                    break
            if is_sensitive:
                beta_ = 30 # NOTE 暫時設定為固定值
            else:
                beta_ = 30 # NOTE 暫時設定為固定值

        capacity = np.random.triangular(400, 800, 1200)

        p = Person(
            person_id = current_id,
            budget = budget,
            capacity = capacity,
            person_type = p_type,
            price_sensitive = is_sensitive,
            favorite_list = fav_list,
            beta = beta_,
            region = store  # 註明此人屬於哪個店地區
        )
        persons.append(p)
        current_id += 1

    return persons, current_id

# -------------------------
# 用來取得 (day, timeslot) 的 lambda
# -------------------------
def get_lambda(day, timeslot):
    is_weekend = (day == 5 or day == 6)
    is_morning = (1 <= timeslot <= 15)
    if is_weekend:
        if is_morning:
            return 25
        else:
            return 40
    else:
        if is_morning:
            return 20
        else:
            return 25

# -------------------------
# Step 6. simulate_static_pricing_for_store
# -------------------------
def simulate_static_pricing_for_store(run_id=1, days=DAYS, store="north", budget_value=1000):
    records = []
    beer_prices = np.array([INIT_PRICE] * NUM_BEERS)
    person_id_counter = 1
    active_persons = []

    for d in range(1, days+1):
        for t in range(1, TIME_SLOTS+1):
            lam = get_lambda(d, t)
            num_new = np.random.poisson(lam)
            new_persons, person_id_counter = generate_new_people_for_store(person_id_counter, num_new, store=store, budget_value=budget_value)
            active_persons.extend(new_persons)

            still_active = []
            for p in active_persons:
                if p.budget <= 0 or p.capacity <= 0:
                    continue

                beer_idx = choose_beer(p, beer_prices)
                price = beer_prices[beer_idx]

                budget_before = p.budget
                capacity_before = p.capacity

                q_demand = demand_function(price, p.beta)
                purchased_volume = min(q_demand, p.capacity, p.budget / price)

                p.capacity -= purchased_volume
                p.budget -= (purchased_volume * price)

                row_data = {
                    "run_id": run_id,
                    "strategy": "STATIC",
                    "store": store,  # 記錄哪個地區店
                    "day": d,
                    "timeslot": t,
                    "person_id": p.person_id,
                    "person_type": p.person_type,
                    "price_sensitive": p.price_sensitive,
                    "region": p.region,
                    "favorite_list": str(p.favorite_list),
                    "beer_idx": beer_idx,
                    "price": price,
                    "demand": q_demand,
                    "purchased_volume": purchased_volume,
                    "budget_before": budget_before,
                    "capacity_before": capacity_before,
                    "budget_after": p.budget,
                    "capacity_after": p.capacity,
                    "revenue": price * purchased_volume,
                    "budget_value": budget_value

                }
                records.append(row_data)
                if p.budget > 0 and p.capacity > 0:
                    still_active.append(p)

            active_persons = still_active

    df = pd.DataFrame(records)
    return df

# -------------------------
# Step 7. simulate_dynamic_pricing_for_store
# -------------------------
def simulate_dynamic_pricing_for_store(run_id=1, days=DAYS, store="north", budget_value=1000):
    records = []
    person_id_counter = 1
    active_persons = []

    # 每個store單獨跑 => 不重設於每天；繼續漲跌
    beer_prices = np.array([INIT_PRICE] * NUM_BEERS)

    for d in range(1, days+1):
        daily_prices = copy.deepcopy(beer_prices)

        for t in range(1, TIME_SLOTS+1):
            beer_volume_sold = np.zeros(NUM_BEERS)

            lam = get_lambda(d, t)
            new_persons, person_id_counter = generate_new_people_for_store(person_id_counter, np.random.poisson(lam), store=store, budget_value=budget_value)
            active_persons.extend(new_persons)

            still_active = []
            for p in active_persons:
                if p.budget <= 0 or p.capacity <= 0:
                    continue

                beer_idx = choose_beer(p, beer_prices)
                price = beer_prices[beer_idx]

                budget_before = p.budget
                capacity_before = p.capacity

                q_demand = demand_function(price, p.beta)
                purchased_volume = min(q_demand, p.capacity, p.budget / price)

                p.capacity -= purchased_volume
                p.budget -= (purchased_volume * price)
                beer_volume_sold[beer_idx] += purchased_volume

                row_data = {
                    "run_id": run_id,
                    "strategy": "DYNAMIC",
                    "store": store,
                    "day": d,
                    "timeslot": t,
                    "person_id": p.person_id,
                    "person_type": p.person_type,
                    "price_sensitive": p.price_sensitive,
                    "region": p.region,
                    "favorite_list": str(p.favorite_list),
                    "beer_idx": beer_idx,
                    "price": price,
                    "demand": q_demand,
                    "purchased_volume": purchased_volume,
                    "budget_before": budget_before,
                    "capacity_before": capacity_before,
                    "budget_after": p.budget,
                    "capacity_after": p.capacity,
                    "revenue": price * purchased_volume,
                    "budget_value": budget_value

                }
                records.append(row_data)

                if p.budget > 0 and p.capacity > 0:
                    still_active.append(p)

            # 每時段結束調整價格
            for i in range(NUM_BEERS):
                vol = beer_volume_sold[i]
                increments = int(vol // 500)
                if increments > 0:
                    beer_prices[i] *= (1 + PRICE_INCREASE) ** increments
                elif vol == 0:
                    beer_prices[i] *= (1 - PRICE_DECREASE)

                # 檢查漲停 / 跌停
                if beer_prices[i] > daily_prices[i] * UPPER_BOUND:
                    beer_prices[i] = daily_prices[i] * UPPER_BOUND
                if beer_prices[i] < daily_prices[i] * LOWER_BOUND:
                    beer_prices[i] = daily_prices[i] * LOWER_BOUND

            active_persons = still_active

        # 不清空 active_persons (依需求)
        active_persons = []

    df = pd.DataFrame(records)
    return df


# -------------------------
# Step 8. 主程式: 執行「北部店 vs 南部店」模擬
# -------------------------
if __name__ == "__main__":
    np.random.seed(42)

    N_SIM = 10  # 若要多次模擬可設大一些
    all_runs = []

    scenarios = [
        {"store":"north","budget_value":800},
        {"store":"north","budget_value":1000},
        {"store":"north","budget_value":1200},
        {"store":"south","budget_value":800},
        {"store":"south","budget_value":1000},
        {"store":"south","budget_value":1200},
    ]

    # 紀錄最終利潤方便查看
    final_profits = []

    for sc in scenarios:
        store = sc["store"]
        budget_val = sc["budget_value"]

        total_static_revenue = 0.0
        total_dynamic_revenue = 0.0

        # 跑 N_SIM 次
        for run_id in range(1, N_SIM+1):
            # 靜態
            df_static = simulate_static_pricing_for_store(
                run_id=run_id, days=7, store=store, budget_value=budget_val
            )
            static_rev = df_static["revenue"].sum()
            total_static_revenue += static_rev

            # 動態
            df_dynamic = simulate_dynamic_pricing_for_store(
                run_id=run_id, days=7, store=store, budget_value=budget_val
            )
            dynamic_rev = df_dynamic["revenue"].sum()
            total_dynamic_revenue += dynamic_rev

            # 合併
            all_runs.append(df_static)
            all_runs.append(df_dynamic)

        avg_static_revenue = total_static_revenue / N_SIM
        avg_dynamic_revenue = total_dynamic_revenue / N_SIM
        print(f"\nScenario: store={store}, budget={budget_val}, repeated {N_SIM} times")
        print(f"   - [Static] avg revenue= {avg_static_revenue:.2f}")
        print(f"   - [Dynamic] avg revenue= {avg_dynamic_revenue:.2f}")

        final_profits.append({
            "store": store,
            "budget_val": budget_val,
            "static_avg_revenue": avg_static_revenue,
            "dynamic_avg_revenue": avg_dynamic_revenue
        })

    final_df = pd.concat(all_runs, ignore_index=True)
    final_df.to_csv("simulation_details_case1.csv", index=False)

    print("\n模擬完成，所有結果存於 simulation_details_case1.csv")
    print("以下為各情境平均營收比較：")
    for row in final_profits:
        print(row)

In [ ]:
"""
plot_results_case1

讀取 simulation_details_case1.csv （其中包含 store, budget_value, strategy, timeslot 等欄位），
繪製幾張圖表，區分北/南部、不同預算(800/1000/1200)、以及 STATIC / DYNAMIC。
"""

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 如果你在主程式跑了 N_SIM=100 次，可能想對 total volume / revenue 做平均 (可視需求)
N_SIM = 10  # 與主程式相同

def plot_average_price_timeslot(csv_file="simulation_details_case1.csv"):
    df = pd.read_csv(csv_file)

    # groupby => 取 price 的 mean
    g = df.groupby(["store", "budget_value", "strategy", "timeslot"], as_index=False)["price"].mean()

    # 改用 sns.relplot(kind="line") + col="store"
    # 注意：relplot 預設會建立 FacetGrid
    gplot = sns.relplot(
        data=g,
        x="timeslot", y="price",
        hue="budget_value", style="strategy",
        kind="line",
        col="store",          # 這裡加: 依 store 分成 2 子圖
        col_wrap=2,          # 若有更多 store，可設定 col_wrap，這裡 2家店 => col_wrap=2
        markers=True, dashes=False,
        height=5, aspect=1.2
    )
    gplot.fig.suptitle("Average Transaction Price vs. Timeslot\n(North vs. South, Different Budgets)", y=1.05)
    gplot.set_axis_labels("Timeslot (1~26)", "Average Price (Yuan)")

    # 另可調整 legend title
    gplot._legend.set_title("Budget / Strategy")

    # 儲存
    gplot.savefig("case1_timeslot_avg_price_facet.png")
    plt.show()


def plot_total_volume_timeslot(csv_file="simulation_details_case2.csv"):
    df = pd.read_csv(csv_file)

    g = df.groupby(["store", "budget_value", "strategy", "timeslot"], as_index=False)["purchased_volume"].sum()
    # 若要平均 => / N_SIM
    g["purchased_volume"] /= N_SIM

    # 用 relplot + col="store" 分成 2 子圖
    gplot = sns.relplot(
        data=g,
        x="timeslot", y="purchased_volume",
        hue="budget_value", style="strategy",
        kind="line",
        col="store", 
        col_wrap=2,
        markers=True, dashes=False,
        height=5, aspect=1.2
    )
    gplot.fig.suptitle("Average Sales Volume vs. Timeslot\n(North vs. South, Different Budgets)", y=1.05)
    gplot.set_axis_labels("Timeslot (1~26)", "Avg Sales Volume (ml)")
    gplot._legend.set_title("Budget / Strategy")

    gplot.savefig("case1_timeslot_volume_facet.png")
    plt.show()


def plot_total_revenue(csv_file="simulation_details_case2.csv"):
    """
    Plot 3: 不同 store/budget_value/strategy 的「總營收」條形圖，
            若要平均 => / N_SIM；可視需求
    """
    df = pd.read_csv(csv_file)

    if "revenue" not in df.columns:
        # 若無 => df["revenue"] = df["price"] * df["purchased_volume"]
        df["revenue"] = df["price"] * df["purchased_volume"]

    # groupby => sum(revenue)
    grouped = df.groupby(["store", "budget_value", "strategy"], as_index=False)["revenue"].sum()
    # 若要看單次模擬平均 => / N_SIM
    grouped["revenue"] /= N_SIM

    plt.figure(figsize=(10,6))
    # 繪圖: x=budget_value, hue="strategy", col="store"? or 直接在一張圖
    # 這裡示範 x="budget_value", hue="strategy", 分 store= row
    # 也可只用 x="budget_value", hue="strategy", col="store"
    
    # 簡單做法：在同一張圖，將 store 也放到 groupby or hue
    # 但常見做法是 => x="budget_value", hue="strategy", col="store"
    # 這裡先示範 "store" 放在 x 軸, "budget_value" 放 hue
    # 或反過來 => x="budget_value", hue="store"
    
    plt.title("Average Total Revenue (North vs. South, Different Budgets, Static/Dynamic)")

    # 如果想將三個維度都放在 barplot => 可做 catplot，但這裡用 barplot + move some to legend
    # 先合併 store & strategy => "store_strategy" 文字
    grouped["store_strategy"] = grouped["store"] + "_" + grouped["strategy"].astype(str)
    # x => budget_value, hue => store_strategy
    sns.barplot(
        data=grouped,
        x="budget_value", y="revenue",
        hue="store_strategy", palette="viridis"
    )
    plt.xlabel("Budget Value")
    plt.ylabel("Avg Revenue")
    plt.legend(title="Store + Strategy")
    plt.tight_layout()
    plt.savefig("case1_total_revenue.png")
    plt.show()

def main():
    csv_file = "simulation_details_case1.csv"
    print(f"Reading {csv_file} and generating plots...")

    plot_average_price_timeslot(csv_file)
    plot_total_volume_timeslot(csv_file)
    plot_total_revenue(csv_file)

    print("All plots saved!")

if __name__ == "__main__":
    main()


## case2

In [ ]:
import numpy as np
import pandas as pd
import copy

class Person:
    """
    - person_id: 全域唯一編號
    - budget: 剩餘預算
    - capacity: 剩餘可喝量 (ml)
    - person_type: 'A' / 'B' / 'C'
    - price_sensitive: True / False
    - favorite_list: 若 B/C 型，紀錄可接受的酒款；A 型可為 None
    - beta: 需求函數中對價格的敏感度
    """
    def __init__(self, person_id, budget, capacity, person_type, price_sensitive, favorite_list, beta):
        self.person_id = person_id
        self.budget = budget
        self.capacity = capacity
        self.person_type = person_type
        self.price_sensitive = price_sensitive
        self.favorite_list = favorite_list
        self.beta = beta

# -------------------------
# Step 2. 全域參數
# -------------------------
NUM_BEERS = 20
INIT_PRICE = 1.0
UPPER_BOUND = 1.2
LOWER_BOUND = 0.5
PRICE_INCREASE = 0.005
PRICE_DECREASE = 0.04
TIME_SLOTS = 26
DAYS = 7

ALPHA = 500   # 基礎需求量
NOISE_STD = 100  # 需求量的雜訊

# 三種類型的比例 (A / B / C)
P_A = 0.4
P_B = 0.3
P_C = 0.3

P_SENSITIVE = 0.5  # 50% 價格敏感

# -------------------------
# (新增) 熱門酒款設定
# -------------------------
HOT_BEERS = [0, 1, 2, 3, 4]  # 假設前 5 款為熱門
# 欲使熱門酒款更容易被抽中，可以提高其權重
# 例如：熱門酒款的權重=3，非熱門酒款權重=1
beer_weights = [3 if i in HOT_BEERS else 1 for i in range(NUM_BEERS)]
weight_sum = sum(beer_weights)

def sample_hot_beers(k):
    """
    從 20 款啤酒中，依照 beer_weights 的權重，抽 k 個 (不重複)。
    回傳: list of beer indices
    """
    # 使用 np.random.choice 時，可傳入 p=加權後的機率
    # 首先計算各酒的機率
    probs = np.array(beer_weights) / weight_sum
    # 不重複抽 k 個
    chosen = np.random.choice(
        np.arange(NUM_BEERS),
        size=k,
        replace=False,
        p=probs
    )
    return chosen.tolist()

def sample_one_hot_beer():
    """
    從 20 款啤酒中，用權重抽 1 個
    """
    probs = np.array(beer_weights) / weight_sum
    return np.random.choice(np.arange(NUM_BEERS), p=probs)

# -------------------------
# Step 3. 需求函數
# -------------------------
def demand_function(price, beta):
    noise = np.random.normal(0, NOISE_STD)
    q = ALPHA - beta * price + noise
    return max(0, q)

# -------------------------
# Step 4. 選酒邏輯 (A/B/C + 價格敏感)
# -------------------------
def choose_beer(person, beer_prices):
    ptype = person.person_type
    sensitive = person.price_sensitive
    flist = person.favorite_list
    all_indices = np.arange(NUM_BEERS)

    # B 型: 單一款
    if ptype == 'B':
        return flist[0]

    # C 型: 多款清單
    if ptype == 'C':
        if sensitive:
            # 挑清單中最便宜
            possible_prices = [beer_prices[b] for b in flist]
            best_idx = flist[np.argmin(possible_prices)]
            return best_idx
        else:
            # 在清單中隨機
            return np.random.choice(flist)

    # A 型: 大雜燴 (可能所有酒都喝)
    if sensitive:
        # (1/price) 為權重
        inv_prices = 1 / beer_prices
        w = inv_prices / inv_prices.sum()
        return np.random.choice(all_indices, p=w)
    else:
        # 純隨機
        return np.random.randint(0, NUM_BEERS)

# -------------------------
# Step 5. 產生新客人 (含 A/B/C + 敏感度)
#    於此步驟中，引入「熱門酒款」的機制
# -------------------------
def generate_new_people(base_person_id, num_people):
    persons = []
    current_id = base_person_id

    for _ in range(num_people):
        # 決定類型 (A/B/C)
        r = np.random.rand()
        if r < P_A:
            p_type = 'A'
        elif r < P_A + P_B:
            p_type = 'B'
        else:
            p_type = 'C'

        # 價格敏感
        is_sensitive = (np.random.rand() < P_SENSITIVE)

        # favorite_list # NOTE
        if p_type == 'A':
            # A 型: 不限定任何固定清單 => 之後選酒才決定
            fav_list = None
        elif p_type == 'B':
            # B 型: 單一款 => 從熱門機制抽 1 (較容易抽到熱門)
            fav_list = [sample_one_hot_beer()]
        else:
            # C 型: 多款清單 => 從熱門機制抽 2~5 款
            L = np.random.randint(2, 6)  # 2~5
            fav_list = sample_hot_beers(L)

        # budget
        while True:
            budget = np.random.normal(1000, 200)
            if budget > 0:
                break

        # capacity
        capacity = np.random.triangular(400, 800, 1200)

        # beta
        if is_sensitive:
            beta_ = 40
        else:
            beta_ = 20

        p = Person(
            person_id=current_id,
            budget=budget,
            capacity=capacity,
            person_type=p_type,
            price_sensitive=is_sensitive,
            favorite_list=fav_list,
            beta=beta_
        )
        persons.append(p)
        current_id += 1

    return persons, current_id


# -------------------------
# (新) Step 5-1. 用來取得 (day, timeslot) 的 lambda (客流量)
# -------------------------
def get_lambda(day, timeslot):
    """
    day=1: 星期一, day=2: 星期二, ...
    day=5: 星期五, day=6: 星期六, day=7: 星期日
    early slot: 1~15
    late slot: 16~26
    """
    is_weekend = (day == 5 or day == 6)
    is_morning = (1 <= timeslot <= 15)

    if is_weekend:
        if is_morning:
            return 25
        else:
            return 40
    else:
        if is_morning:
            return 20
        else:
            return 25



# -------------------------
# 2. 新增營業成本、可變營業時數
# -------------------------
COST_PER_SLOT = 3000  # 每半小時營業固定成本
DEFAULT_OPEN_SLOTS = 26  # 全部營業

def simulate_static_pricing(run_id=1, days=7, open_slots=26, start_slot=1):
    """
    open_slots: 實際營業的 time slots 數 (default=26)
    其餘邏輯與原本相同，但只跑到 open_slots
    """
    records = []
    beer_prices = np.array([1.0]*NUM_BEERS)
    person_id_counter = 1
    active_persons = []

    for d in range(1, days+1):
        for t in range(1, open_slots+1):
            # 映射到實際slot
            actual_slot = start_slot + t - 1
            
            lam = get_lambda(d, actual_slot)
            num_new = np.random.poisson(lam)
            new_persons, person_id_counter = generate_new_people(person_id_counter, num_new)
            active_persons.extend(new_persons)

            still_active = []
            for p in active_persons:
                if p.budget <= 0 or p.capacity <= 0:
                    continue

                beer_idx = choose_beer(p, beer_prices)
                price = beer_prices[beer_idx]

                q_demand = demand_function(price, p.beta)
                purchased_volume = min(q_demand, p.capacity, p.budget / price)

                budget_before = p.budget
                capacity_before = p.capacity

                p.capacity -= purchased_volume
                p.budget -= (purchased_volume * price)

                row_data = {
                    "run_id": run_id,
                    "strategy": "STATIC",
                    "day": d,
                    "timeslot": actual_slot,
                    "person_id": p.person_id,
                    "person_type": p.person_type,
                    "price_sensitive": p.price_sensitive,
                    "favorite_list": str(p.favorite_list),
                    "beer_idx": beer_idx,
                    "price": price,
                    "demand": q_demand,
                    "purchased_volume": purchased_volume,
                    "budget_before": budget_before,
                    "capacity_before": capacity_before,
                    "budget_after": p.budget,
                    "capacity_after": p.capacity
                }
                records.append(row_data)

                if p.budget > 0 and p.capacity > 0:
                    still_active.append(p)
            active_persons = still_active

        # 不保留跨日
        active_persons = []

    df = pd.DataFrame(records)
    return df

def simulate_dynamic_pricing(run_id=1, days=7, open_slots=26, start_slot=1):
    records = []
    person_id_counter = 1
    active_persons = []

    beer_prices = np.array([1.0] * NUM_BEERS)

    for d in range(1, days+1):
        daily_prices = copy.deepcopy(beer_prices)

        for t in range(1, open_slots+1):
            beer_volume_sold = np.zeros(NUM_BEERS)
            actual_slot = start_slot + t - 1
            lam = get_lambda(d, actual_slot)
            num_new = np.random.poisson(lam)
            new_persons, person_id_counter = generate_new_people(person_id_counter, num_new)
            active_persons.extend(new_persons)

            still_active = []
            for p in active_persons:
                if p.budget <= 0 or p.capacity <= 0:
                    continue

                beer_idx = choose_beer(p, beer_prices)
                price = beer_prices[beer_idx]

                q_demand = demand_function(price, p.beta)
                purchased_volume = min(q_demand, p.capacity, p.budget / price)

                budget_before = p.budget
                capacity_before = p.capacity

                p.capacity -= purchased_volume
                p.budget -= purchased_volume * price
                beer_volume_sold[beer_idx] += purchased_volume

                row_data = {
                    "run_id": run_id,
                    "strategy": "DYNAMIC",
                    "day": d,
                    "timeslot": actual_slot,
                    "person_id": p.person_id,
                    "person_type": p.person_type,
                    "price_sensitive": p.price_sensitive,
                    "favorite_list": str(p.favorite_list),
                    "beer_idx": beer_idx,
                    "price": price,
                    "demand": q_demand,
                    "purchased_volume": purchased_volume,
                    "budget_before": budget_before,
                    "capacity_before": capacity_before,
                    "budget_after": p.budget,
                    "capacity_after": p.capacity
                }
                records.append(row_data)

                if p.budget > 0 and p.capacity > 0:
                    still_active.append(p)

            # 動態價格調整
            for i in range(NUM_BEERS):
                vol = beer_volume_sold[i]
                increments = int(vol // 500)
                if increments > 0:
                    beer_prices[i] *= (1 + PRICE_INCREASE) ** increments
                elif vol == 0:
                    beer_prices[i] *= (1 - PRICE_DECREASE)

                # 漲停 / 跌停
                if beer_prices[i] > daily_prices[i] * UPPER_BOUND:
                    beer_prices[i] = daily_prices[i] * UPPER_BOUND
                if beer_prices[i] < daily_prices[i] * LOWER_BOUND:
                    beer_prices[i] = daily_prices[i] * LOWER_BOUND

            active_persons = still_active

        active_persons = []

    df = pd.DataFrame(records)
    return df

if __name__=="__main__":
    np.random.seed(42)

    N_SIM = 1000    # 這裡改成要模擬幾次
    config_list = [
        {"open_slots": 26, "start_slot": 1},   # 全日(11:00-00:00)
        {"open_slots": 18, "start_slot": 9},   # 15:00開始營業8小時 (slot=9~24 => 15:00~24:00)
        {"open_slots": 14, "start_slot": 13},  # 17:00開始營業7小時 (slot=13~26 => 17:00~00:00)
    ]
    DAYS = 7
    all_runs = []

    for conf in config_list:
        open_slots = conf["open_slots"]
        start_slot = conf["start_slot"]

        for rep in range(1, N_SIM+1):
            # 模擬 靜態定價
            df_static = simulate_static_pricing(run_id=rep, days=DAYS, open_slots=open_slots, start_slot=start_slot)

            # 模擬 動態定價
            df_dynamic = simulate_dynamic_pricing(run_id=rep, days=DAYS, open_slots=open_slots, start_slot=start_slot)

            # 計算營收
            df_static["revenue"] = df_static["price"] * df_static["purchased_volume"]
            df_dynamic["revenue"] = df_dynamic["price"] * df_dynamic["purchased_volume"]

            total_static_revenue = df_static["revenue"].sum()
            total_dynamic_revenue = df_dynamic["revenue"].sum()

            # 計算營業成本
            daily_cost = open_slots * COST_PER_SLOT
            total_cost = daily_cost * DAYS

            net_static = total_static_revenue - total_cost
            net_dynamic = total_dynamic_revenue - total_cost

            print(f"\n[open_slots={open_slots}, start_slot={start_slot}] - 第 {rep} 次模擬，天數={DAYS}")
            print(f" - 每日成本= {daily_cost}, 7天總成本= {total_cost}")
            print(f" - [STATIC] 營收= {total_static_revenue:.2f}, 淨利= {net_static:.2f}")
            print(f" - [DYNAMIC] 營收= {total_dynamic_revenue:.2f}, 淨利= {net_dynamic:.2f}")

            # 在 DataFrame 裡面註記
            df_static["open_slots"] = open_slots
            df_static["start_slot"] = start_slot
            df_static["cost"] = total_cost
            df_static["net_profit"] = net_static
            df_static["rep_id"] = rep  # 第幾次模擬

            df_dynamic["open_slots"] = open_slots
            df_dynamic["start_slot"] = start_slot
            df_dynamic["cost"] = total_cost
            df_dynamic["net_profit"] = net_dynamic
            df_dynamic["rep_id"] = rep

            all_runs.append(df_static)
            all_runs.append(df_dynamic)

    final_df = pd.concat(all_runs, ignore_index=True)
    final_df.to_csv("simulation_details_case2.csv", index=False)
    print("\n所有模擬完成，結果輸出：simulation_details_case2.csv\n")

In [ ]:
"""
plot_results_case2

Read multiple sets of (open_slots, start_slot) test results from simulation_details_case3.csv,
which might have run multiple times (N_SIM).
Plot simple charts:
   1) net profit bar chart
   2) average price line chart for each timeslot
   3) total sales volume line chart for each timeslot
with "average" (if multiple runs exist).
"""

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 若你在主程式中重複跑了 N_SIM 次，請在這裡設定：
N_SIM = 1000  # or 10, 100, etc.

def plot_net_profit(csv_file="simulation_details_case2.csv"):
    """
    Plot: Net profit bar chart under different (open_slots, start_slot) and (strategy),
          taking average if multiple runs are present.
    """
    df = pd.read_csv(csv_file)
    # 如果主程式沒有記錄 run_id / rep_id，就表示所有資料都直接疊加了 N_SIM times
    
    # Step 1: 如果 CSV 裡 "net_profit" 已經是每次模擬獨立計算的話
    #         我們可以直接 groupby["open_slots","start_slot","strategy"] 做 mean()
    #         代表對 net_profit 做平均
    if "net_profit" in df.columns:
        grouped = df.groupby(["open_slots", "start_slot", "strategy"], as_index=False)["net_profit"].mean()
    else:
        # 如果 CSV 裡面只有 revenue, cost => net_profit = sum(revenue) - unique cost
        # cost 在同 run 內應相同，但對多次模擬需要先 sum(revenue), sum(cost?), or do 'mean(cost)'.
        
        grouped = df.groupby(["open_slots", "start_slot", "strategy"], as_index=False).agg({
            "revenue": "sum",
            "cost": "sum",  # or mean
        })
        # total revenue / N_SIM => average revenue
        grouped["revenue"] = grouped["revenue"] / N_SIM
        # same for cost
        grouped["cost"] = grouped["cost"] / N_SIM
        grouped["net_profit"] = grouped["revenue"] - grouped["cost"]
    
    plt.figure(figsize=(10,6))
    sns.barplot(
        data=grouped,
        x="open_slots", y="net_profit",
        hue="strategy", palette="magma"
    )
    plt.title("Net Profit Comparison (Avg over N_SIM runs) - Different Business Hours, Start Slots")
    plt.xlabel("open_slots (#Business Slots)")
    plt.ylabel("Net Profit (Average)")
    plt.legend(title="Strategy")
    plt.tight_layout()
    plt.savefig("case2_net_profit_bar.png")
    plt.show()

def plot_avg_price_timeslot(csv_file="simulation_details_case2.csv"):
    """
    Plot: Average price line chart for each timeslot.
          If multiple runs are in the CSV, we do groupby -> mean for all runs.
    """
    df = pd.read_csv(csv_file)
    # groupby (open_slots, start_slot, strategy, timeslot) => price.mean()
    # 這裡 mean() 會把多次模擬的交易全部湊在一起平均 => 近似"整體平均價格"
    g = df.groupby(["open_slots", "start_slot", "strategy", "timeslot"], as_index=False)["price"].mean()

    plt.figure(figsize=(12,8))
    sns.lineplot(
        data=g,
        x="timeslot",
        y="price",
        hue="open_slots",
        style="strategy",
        markers=True,
        dashes=False
    )
    plt.title("Average Price per Timeslot (Across N_SIM runs)")
    plt.xlabel("timeslot (1~26)")
    plt.ylabel("Avg Price")
    plt.legend(title="slots / strategy")
    plt.tight_layout()
    plt.savefig("case2_timeslot_price_line.png")
    plt.show()

def plot_total_volume_timeslot(csv_file="simulation_details_case2.csv"):
    """
    Plot: total purchased_volume for each timeslot, grouping all runs => sum, then / N_SIM => average
    """
    df = pd.read_csv(csv_file)
    # groupby => sum purchased_volume
    grouped = df.groupby(["open_slots", "strategy", "timeslot"], as_index=False)["purchased_volume"].sum()
    # 再 / N_SIM => average
    grouped["purchased_volume"] = grouped["purchased_volume"] / N_SIM

    plt.figure(figsize=(12,8))
    sns.lineplot(
        data=grouped,
        x="timeslot", y="purchased_volume",
        hue="open_slots", style="strategy",
        markers=True, dashes=False
    )
    plt.title("Avg Sales Volume per Timeslot (Across N_SIM runs) - Different Hours & Strategies")
    plt.xlabel("timeslot (1~26)")
    plt.ylabel("Purchased Volume (ml)")
    plt.legend(title="slots / strategy")
    plt.tight_layout()
    plt.savefig("case2_timeslot_volume_line.png")
    plt.show()

def main():
    csv_file = "simulation_details_case2.csv"
    print("Reading:", csv_file)
    
    # 1) Net profit bar chart
    plot_net_profit(csv_file)

    # 2) Average price line chart by timeslot
    plot_avg_price_timeslot(csv_file)

    # 3) Total(average) volume line chart by timeslot
    plot_total_volume_timeslot(csv_file)

    print("Plotting completed. Files have been output.")

if __name__=="__main__":
    main()
